In [18]:
from toqito.channels import partial_trace
from qutip import *
from PIL import Image
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from scipy import linalg
import math
import matplotlib.pyplot as plt
from scipy import optimize
import random
from math import *
import pandas as pd
import random
from mpl_toolkits.mplot3d import Axes3D
from sys import stdout
from tqdm import tqdm
from tqdm import trange
from scipy.linalg import fractional_matrix_power

In [34]:
import time
from datetime import datetime as dt 
date = dt.now()
printdate = date.strftime('%Y%m%d_%H%M%S')
print(date)

2023-06-21 20:28:29.612690


In [19]:
def state_fidelity(rho_1, rho_2): #fidelity
        if np.shape(rho_1) != np.shape(rho_2):
            print("Dimensions of two states do not match.")
            return 0
        else:
            sqrt_rho_1 = fractional_matrix_power(rho_1, 1 / 2)
            fidelity = np.trace(fractional_matrix_power(sqrt_rho_1 @ rho_2 @ sqrt_rho_1, 1 / 2)) ** 2
            return np.real(fidelity)

In [20]:
def UO(B1,B2,a,D1,D2):
    i   = 1j
    gamma = 2*pi*2.8
    D     = 2870
    UA = [[(B2**2+B1**2*cos(a))/(B1**2+B2**2), -i*B1*(e**(-i*D1))*sin(a)/sqrt(B1**2+B2**2), ((-1+cos(a))*B1*B2*(e**(-i*(D1-D2))))/(B1**2+B2**2)],
            [-i*B1*(e**(i*D1))*sin(a)/sqrt(B1**2+B2**2), cos(a), -i*B2*(e**(i*D2))*sin(a)/sqrt(B1**2+B2**2)],
            [((-1+cos(a))*B1*B2*e**(i*(D1-D2)))/(B1**2+B2**2), -i*B2*(e**(-i*D2))*sin(a)/sqrt(B1**2+B2**2), (B1**2+B2**2*cos(a))/(B1**2+B2**2)]]
    return UA

## Define dimension, pauli matrices
i   = 1j #1j
sx  = 1/sqrt(2)*np.array([[0, 1, 0],[1, 0, 1], [0, 1, 0]])
sy  = 1/sqrt(2)/i*np.array([[0, 1, 0], [-1, 0, 1],[0, -1, 0]])
sz  = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
#sz  = [1, 0, 0; 0, -1, 0; 0, 0, 0]
I   = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Rotation matrix projected into 2 level system
Sxp  = np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]])
Sxm  = np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]])
Syp  = 1/i*np.array([[0, 1, 0], [-1, 0, 0], [0, 0, 0]])
Sym  = 1/i*np.array([[0, 0, 0], [0, 0, 1], [0, -1, 0]])
Szp  = np.array([[1, 0, 0], [0, -1, 0], [0, 0, 0]])

#Gellman matrix
Sx  = np.array([[0, 0, 1],[0, 0, 0], [1, 0, 0]])
Sy  = np.array([[0, 0, -i],[0, 0, 0], [i, 0, 0]])
Sz  = np.array([[1, 0, 0],[0, 0, 0], [0, 0, -1]])

# Pauli basis for 13C nuclear spin
Ix  = 1/2*np.array([[0, 0, 1], [0, 0, 0], [1, 0, 0]])   
Iy  = 1/2/i*np.array([[0, 0, 1], [0, 0, 0], [-1, 0, 0]])
Iz  = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])


## Define sweep parameters
Sweep = 1001
N = Sweep
B = 403 #[G] magnetic field

T = 5; # sweep tau [us]
t = np.linspace(0,T,N)
n = 32; # number of pi pulses

## Define gate operations
# Single Q ms=+1
U090xp = UO(1,0,pi/4,0,0)
U090xmp = UO(1,0,-pi/4,0,0)
U090yp = UO(1,0,pi/4,pi/2,0)
U090ymp = UO(1,0,-pi/4,pi/2,0)
U180xp = UO(1,0,pi/2,0,0)
U180xmp = UO(1,0,-pi/2,0,0)

#Single Q ms=-1
U090xm = UO(0,1,pi/4,0,0)
U090xmm = UO(0,1,-pi/4,0,0)
U180xm = UO(0,1,pi/2,0,0)
U180xmm = UO(0,1,pi/2,0,0)

# Define initial state of the system

irho_p = np.array([[1,0,0],[0,0,0],[0,0,0]]) #[0,0,0;0,0,0]

irho_m = np.array([[0,0,0],[0,0,0],[0,0,1]]) #[0,0,0;0,0,1]

irho_z = np.array([[0,0,0],[0,1,0],[0,0,0]]) #[0,1,0;0,0,0]

irho_mix = np.array([[1/2,0,0],[0,1/2,0],[0,0,0]]) #[1/2,0,0;0,1/2,0;0,0,0]

irho_Z = np.array([[0,0,0],[0,0,0],[0,0,1]]) #target state

irho_MIX = np.array([[1/2,0,0],[0,0,0],[0,0,1/2]])

In [21]:
arr1 = []

In [22]:
irho = np.kron(irho_z,irho_MIX) #initial state
print(irho)

[[0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.5 0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0. ]]


In [23]:
trace = [1, 1, 0, 100, 100, 1000]
vvv = [0, 0, 0, 0]
normalxyz = [0, 0, 0]

#for making 13C nuclear random dataset
gammaN = 2*pi*1.071e-3 #[MHz/G]
# Al    = 2*pi * random.uniform(0.05, 0.8) #[MHz] # A_|| hyperfine term
# Ap = 2*pi* random.uniform(0.05, 0.3) #[MHz] # A_per hyperfine term

Al = 1.1876071666049706
Ap = 1.3050296350856783

#Initialization
rho_0 = (np.kron(U090xp,I))@irho@((np.kron(U090xp,I)).conj().T) # superposition state on NV

Sa= []

ham = Al*np.kron(sz,Iz) + Ap*np.kron(sz,Ix) + B*gammaN*np.kron(I,Iz) # Hamiltonian
eigvals = np.linalg.eigh(ham)[0]            # diagonalizing the Hamiltonian 여기서부터 문제 
eigvecs = -1*np.linalg.eigh(ham)[1]         # eigenvectors
E = np.diag(eigvals)                        # exponent of eigenvalues
U_H= eigvecs.conj().T                       # unitary matrix formed by eigenvectors

In [24]:
udf = pd.DataFrame(U_H)
udf

,0,1,2,3,4,5,6,7,8
0,0.082803,-0.0,-0.996566,0.0,0.0,0.0,0.000000,0.0,-0.000000
1,-0.000000,-0.0,-0.000000,-0.0,-0.0,-1.0,-0.000000,-0.0,-0.000000
2,0.000000,0.0,0.000000,0.0,0.0,0.0,0.200861,-0.0,0.979620
3,-0.000000,-0.0,-0.000000,-0.0,-1.0,-0.0,-0.000000,-0.0,-0.000000
4,-0.000000,1.0,-0.000000,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.000000
5,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.0,-0.000000,-1.0,-0.000000
6,-0.000000,-0.0,-0.000000,-0.0,-0.0,-0.0,0.979620,-0.0,-0.200861
7,-0.000000,-0.0,-0.000000,-1.0,-0.0,-0.0,-0.000000,-0.0,-0.000000
8,0.996566,-0.0,0.082803,-0.0,-0.0,-0.0,-0.000000,-0.0,-0.000000


In [25]:
rho1 = np.kron(U090yp,I)@irho@(np.kron(U090yp,I).conj().T)                              # Ry 90도
df1 = pd.DataFrame(rho1)
df1 

,0,1,2,3,4,5,6,7,8
0,0.25-0.00j,0.0+0.0j,0.00+0.00j,-0.25-0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
1,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
2,0.00+0.00j,0.0+0.0j,0.25-0.00j,0.00+0.00j,0.0+0.0j,-0.25-0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
3,-0.25+0.00j,0.0+0.0j,0.00+0.00j,0.25+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
4,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
5,0.00+0.00j,0.0+0.0j,-0.25+0.00j,0.00+0.00j,0.0+0.0j,0.25+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
6,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
7,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
8,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.00+0.00j,0.0+0.0j,0.00+0.00j,0.0+0.0j,0.0+0.0j,0.0+0.0j


In [26]:
vari=[1.4120767867009363,22.0,0.2633900964243493,22.0]  #초기값

In [27]:
U_e2=(U_H.conj().T)@(linalg.expm(-i*E* vari[0]/2)@U_H)                                  # for tau/2
U_e=(U_H.conj().T)@(linalg.expm(-i*E* vari[0])@U_H)                                     # for tau
rho2=U_e2@rho1@(U_e2.conj().T)                                                          # first tau/2
for k in range(1,2*math.trunc(vari[1])):                                                # N과 tau를 N개 생성
    rho2 = U_e@np.kron(U180xp,I) @ rho2 @ (np.kron(U180xp,I).conj().T) @ (U_e.conj().T) # N & tau
rho3 = U_e2 @ np.kron(U180xp,I) @ rho2 @ (np.kron(U180xp,I).conj().T) @ (U_e2.conj().T) # last N & tau/2

In [28]:
# df2 = pd.DataFrame(rho2)
df2 = pd.DataFrame(rho3)
df2

,0,1,2,3,4,5,6,7,8
0,2.500000e-01-3.594832e-17j,0.0+0.0j,-2.590329e-16+4.865103e-17j,4.165535e-03-4.775084e-03j,0.0+0.0j,8.433491e-03+2.497773e-01j,0.0+0.0j,0.0+0.0j,0.0+0.0j
1,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
2,-3.011204e-16-9.248602e-17j,0.0+0.0j,2.500000e-01-6.772246e-17j,-8.433491e-03+2.497773e-01j,0.0+0.0j,4.165535e-03+4.775084e-03j,0.0+0.0j,0.0+0.0j,0.0+0.0j
3,4.165535e-03+4.775084e-03j,0.0+0.0j,-8.433491e-03-2.497773e-01j,2.500000e-01-1.669181e-16j,0.0+0.0j,3.074759e-16+2.837234e-17j,0.0+0.0j,0.0+0.0j,0.0+0.0j
4,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
5,8.433491e-03-2.497773e-01j,0.0+0.0j,4.165535e-03-4.775084e-03j,2.763787e-16-1.488200e-17j,0.0+0.0j,2.500000e-01-2.673152e-17j,0.0+0.0j,0.0+0.0j,0.0+0.0j
6,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
7,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j
8,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.000000e+00+0.000000e+00j,0.0+0.0j,0.0+0.0j,0.0+0.0j


In [29]:
#for e Rx(pi/2)
rho4 = np.kron(U090xp,I)@rho3@(np.kron(U090xp,I).conj().T)                              # Rx 90도
df4 = pd.DataFrame(rho4)
print(df4)

                            0         1                           2  \
0  2.547751e-01-1.157567e-16j  0.0+0.0j -2.497773e-01+8.433491e-03j   
1  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
2 -2.497773e-01-8.433491e-03j  0.0+0.0j  2.452249e-01+5.768255e-18j   
3  4.165535e-03+1.129404e-16j  0.0+0.0j -1.677551e-17+2.606712e-16j   
4  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
5 -1.169552e-17+2.941752e-16j  0.0+0.0j  4.165535e-03-3.135056e-17j   
6  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
7  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   
8  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j   

                            3         4                           5         6  \
0  4.165535e-03-1.534143e-16j  0.0+0.0j -3.800253e-17-3.309030e-16j  0.0+0.0j   
1  0.000000e+00+0.000000e+00j  0.0+0.0j  0.000000e+00+0.000000e+00j  0.0+0.0j   
2  6.639257e-17-2.997801e-16j  0.0+0.0j  4.165

In [30]:
U_e2=(U_H.conj().T)@(linalg.expm(-i*E*vari[2]/2)@U_H)                                   # for tau/2
U_e=(U_H.conj().T)@(linalg.expm(-i*E*vari[2])@U_H)                                      # for tau/2
rho5=U_e2@rho4@(U_e2.conj().T)                                                          # first tau/2
for k in range(1,2*math.trunc(vari[3])):                                                # N과 tau를 N개 생성
    rho5 = U_e@np.kron(U180xp,I) @ rho5 @ (np.kron(U180xp,I).conj().T) @ (U_e.conj().T) # N & tau
rho6 = U_e2 @ np.kron(U180xp,I) @ rho5 @ (np.kron(U180xp,I).conj().T) @ (U_e2.conj().T) # last N & tau/2

df6 = pd.DataFrame(rho6)
print(df6)

                    0         1                   2                   3  \
0  0.241323-0.000000j  0.0+0.0j -0.008724+0.249662j  0.004158-0.000019j   
1  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
2 -0.008724-0.249662j  0.0+0.0j  0.258677+0.000000j -0.000174+0.000185j   
3  0.004158+0.000019j  0.0+0.0j -0.000174-0.000185j  0.279915-0.000000j   
4  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
5  0.000174-0.000185j  0.0+0.0j  0.004158-0.000019j  0.012568+0.247850j   
6  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
7  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
8  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   

          4                   5         6         7         8  
0  0.0+0.0j  0.000174+0.000185j  0.0+0.0j  0.0+0.0j  0.0+0.0j  
1  0.0+0.0j  0.000000+0.000000j  0.0+0.0j  0.0+0.0j  0.0+0.0j  
2  0.0+0.0j  0.004158+0.000019j  0.0+0.0j  0.0+0.0j  0.0+

In [31]:
U_e2=(U_H.conj().T)@(linalg.expm(-i*E* vari[0]/2)@U_H)                                  # for tau/2
U_e=(U_H.conj().T)@(linalg.expm(-i*E* vari[0])@U_H)                                     # for tau
rho7=U_e2@rho6@(U_e2.conj().T)                                                          # first tau/2
for k in range(1,2*math.trunc(vari[1])):                                                # N과 tau를 N개 생성
    rho7 = U_e@np.kron(U180xp,I) @ rho7 @ (np.kron(U180xp,I).conj().T) @ (U_e.conj().T) # N & tau
rho8 = U_e2 @ np.kron(U180xp,I) @ rho7 @ (np.kron(U180xp,I).conj().T) @ (U_e2.conj().T) # last N & tau/2

df8 = pd.DataFrame(rho8)
print(df8)

                    0         1                   2                   3  \
0  0.000430+0.000000j  0.0+0.0j -0.005339-0.012991j  0.000110+0.000064j   
1  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
2 -0.005339+0.012991j  0.0+0.0j  0.499570-0.000000j -0.000039-0.004163j   
3  0.000110-0.000064j  0.0+0.0j -0.000039+0.004163j  0.001411+0.000000j   
4  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
5  0.000039+0.004163j  0.0+0.0j  0.000110+0.000064j -0.002007+0.026119j   
6  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
7  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   
8  0.000000+0.000000j  0.0+0.0j  0.000000+0.000000j  0.000000+0.000000j   

          4                   5         6         7         8  
0  0.0+0.0j  0.000039-0.004163j  0.0+0.0j  0.0+0.0j  0.0+0.0j  
1  0.0+0.0j  0.000000+0.000000j  0.0+0.0j  0.0+0.0j  0.0+0.0j  
2  0.0+0.0j  0.000110-0.000064j  0.0+0.0j  0.0+0.0j  0.0+

In [40]:
partial_trace(rho8, 1)

array([[ 0.00184078+1.46592668e-16j,  0.        +0.00000000e+00j,
        -0.00734575-3.91103223e-02j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j],
       [-0.00734575+3.91103223e-02j,  0.        +0.00000000e+00j,
         0.99815922-3.35141946e-16j]])

In [41]:
irho_Z

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 1]])

In [35]:
df8.to_csv('C:/Users/Administrator/git_zorocrit/qhackathon-2023/matrix/' + printdate + '_compare.csv',index=False)

In [42]:
state_fidelity(partial_trace(rho8, 1), irho_Z)

0.99815921608879